[`Magma`](https://github.com/phanrahan/magma) is a simple hardware construction languages written `Python 3`. The central abstraction in `Magma` is a `Circuit`. Magma circuits are analagous to verilog modules. A circuit is a set of functional units that are wired together. Although wiring modules together may seem low-level, it encourages hardware designers to build reusable components, similar to how programmers build libraries. 

`Magma` is designed to work with [`Mantle`](https://github.com/phanrahan/mantle). `Mantle` is a library of hardware building blocks such as logic and arithmetic units, registers, and memories. 

The other system is [`Loam`](https://github.com/phanrahan/loam). `Loam` is used to represent *parts* and *boards*. It makes is easy to setup a board such as the Icestick.

Let's start with a simple example, a program that blinks an LED on the Icestick Board.

![icestick](images/icestick.jpg)

First, we import `Magma` as the module m and import an n-bit `Counter` from `Mantle`'

In [1]:
import os
os.environ['MANTLE_TARGET'] = 'ice40'

import magma as m
from mantle import Counter

import mantle lattice ice40
import mantle lattice mantle40


The next step is to setup the IceStick board. We import the class `IceStick` from loam. We then create an instance of an Icestock board, and then turn on the Clock and the LED D5. Turning on the clock and the LED sets up the build environment to use the associated pins on the board.

In [4]:
from loam.boards.icestick import IceStick

# Create an instance of an IceStick board
icestick = IceStick()

# Turn on the Clock 
#  The clock is turned on because we are using a synchronous counter
icestick.Clock.on()

# Turn on the LED D5
icestick.D5.on();

Now that the setup is done, we create a `main` program that runs on the Lattice ICE40 FPGA. This main program becomes the top level module.

Once that is done we create a 22-bit counter, and wire the most-significant bit to the LED. The crystal connected to the ICE40 runs at 12 Mhz. A 22-bit counter holds numbers up to 4 million, so the LED will blink roughly 3 times per second.

In [ ]:
# Define a Magma Circuit on the FPGA on the IceStick
main = icestick.DefineMain()

# Create a 22-bit counter
N = 22
counter = Counter(N)

# Wire bit 21 of the counter to D5.
#  Since the clock frequency is 12Mhz,
#  bit 22 will toggle  approximately once per second
wire(counter.O[N-1], main.D5)

EndDefine()

We then compile the program to verilog. This step also creates a PCF (physical constraints file).

In [ ]:
m.compile('build/blink', main)

Now we run the open source tools for the Lattice ICE40. 
`yosys` synthesizes the verilog file (`blink.v`) and produces a netlist (`blink.blif`).
`arachne-pnr` runs the place and router.
Finally, `icepack` creates the bitstream file.

In [ ]:
%%bash
yosys -q -p 'synth_ice40 -top main -blif build/blink.blif' build/blink.v
arachne-pnr -q -d 1k -o build/blink.txt -p build/blink.pcf build/blink.blif 
icepack build/blink.txt build/blink.bin

Upload the bitstream file (`blink.bin`) to the FPGA. 
The Lattice board needs to be connected to your laptop.

In [ ]:
%%bash 
iceprog build/blink.bin